In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import ujson as json
import glob as g
import random as rnd
import os.path as op
import os
import codecs
import shutil
import time
import math
import numpy as np
from skimage import io, transform
import skimage.color as skc

from tqdm import tqdm

import pandas as pd
import seaborn as sns

import multiprocessing
import hashlib as hl
import scipy.spatial.distance as sd
import tinydb as tb
import re

import subprocess as sp

In [15]:
def split_batch_name(batch):
    match = re.match('(\D+)(.*)',batch)
    return match.groups()
print split_batch_name('SB002/23')

('SB', '002/23')


In [16]:
tests_path = '/media/jakob/bigdata/train_sets'
text_db_path = '/media/jakob/bigdata/train/db.json'
catalogs_dir = '/media/jakob/bigdata/train/'

In [17]:
data = pd.read_csv('/media/jakob/bigdata/train/sample_trainAll.csv')['0']
test = pd.read_csv('/media/jakob/bigdata/train/sample_reproducibility.csv')['0']
sample_set = set(data)
test_set = set(test)

In [20]:
def create_testbed(test_path, text_db_path,  ext = '.png'):
    db = tb.TinyDB(text_db_path)
    if not op.isdir(test_path):
        os.makedirs(test_path)
    samples = db.table('lines')
    for line in tqdm(samples.all()):
        if not line['id'] in test_set:
            cat_name, _ = split_batch_name(line['id'])
            cat_dir     = op.join(test_path,cat_name)
            if not op.isdir(cat_dir):
                os.makedirs(cat_dir)
            line_img = op.join(catalogs_dir, cat_name, line['id'])
            test_out = op.join(cat_dir, line['id'].replace('/', '_'))
            shutil.copy(line_img + ext, test_out + ext)
            with codecs.open(test_out + '.gt.txt', 'w', encoding='utf8') as gt_file:
                gt_file.write(line['gt'])
    db.close()


In [21]:
create_testbed(op.join(tests_path, 'ocropy_all'), text_db_path, ext='.bin.png')

100%|██████████| 400/400 [00:03<00:00, 109.95it/s]
